In [1]:
from typing import *
from crimson.intelli_type import IntelliType
from inspect import getsource
from script import Path
import inspect

from text_editor import * 

In [2]:
import yaml

In [3]:
json_object = {"pathMap" : {
    "path": "Paht",
    "file": "File",
    "ListItem": [1, 2, 3]
}}

print(yaml.dump(json_object))

pathMap:
  ListItem:
  - 1
  - 2
  - 3
  file: File
  path: Paht



In [9]:
class DocWithLink:
    """
    It uses *path*<link/> and *file*<link/>
    
    ``` yaml
    link_map:
        path: Path
        file: File

    ```
    
    """

In [10]:
doc_with_link = remove_indent(DocWithLink.__doc__)

print(doc_with_link)


It uses *path*<link/> and *file*<link/>

``` yaml
link_map:
    path: Path
    file: File

```




In [15]:
import script


script.__dict__["Path"].__doc__

'\n    It is a path.\n    '

In [202]:
from script import Path
from pydantic import BaseModel, Field

def get_location(obj):

    file_path = inspect.getfile(obj)
    
    source_lines, start_line = inspect.getsourcelines(obj)
    
    end_line = start_line + len(source_lines) - 1
    
    location = {
        "file_path": file_path,
        "start_line": start_line,
        "end_line": end_line
    }

    return location

class Location(BaseModel):
    file_path: str = Field(..., description="The file path of the location")
    start_line: int = Field(..., ge=1, description="The starting line number")
    end_line: int = Field(..., ge=1, description="The ending line number")
    optional: Optional[str] = Field(None, ge=1)


In [203]:
import pydantic

location = Location(file_path="/path/to/file.py", start_line=10, end_line=20)


location.model_fields

{'file_path': FieldInfo(annotation=str, required=True, description='The file path of the location'),
 'start_line': FieldInfo(annotation=int, required=True, description='The starting line number', metadata=[Ge(ge=1)]),
 'end_line': FieldInfo(annotation=int, required=True, description='The ending line number', metadata=[Ge(ge=1)]),
 'optional': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, metadata=[Ge(ge=1)])}

In [145]:
location.model_fields["optional"].default

In [199]:
from pydantic import *
from pydantic_core import *

class Empty:
    pass

class ArgProps(BaseModel):
    arg_name: str
    required: bool
    description: str | None
    default: Any | Empty
    annotation: type | Any

    model_config = ConfigDict(arbitrary_types_allowed=True)
    
class DataProps(BaseModel):
    data_name: str
    arg_props_list: List[ArgProps]

In [205]:
def convert_default(field: pydantic.fields.FieldInfo) -> Any|Empty:
    if hasattr(field, "default"):
        if isinstance(field.default, PydanticUndefinedType):
            return Empty
        else:
            field.default
    else:
        return Empty

def get_arg_props(arg_name, field: pydantic.fields.FieldInfo) -> ArgProps:
    arg_props = ArgProps(
        arg_name=arg_name,
        required=field.is_required(),
        default=convert_default(field),
        description=field.description,
        annotation=field.annotation.__name__
    )
    return arg_props

def get_arg_props_list(model_fields: Dict[str, pydantic.fields.FieldInfo]) -> List[ArgProps]:
    arg_props_list = [get_arg_props(arg_name, field) for arg_name, field in model_fields.items()]
    return arg_props_list

In [206]:
arg_props_list = get_arg_props_list(location.model_fields)
arg_props_list

[ArgProps(arg_name='file_path', required=True, description='The file path of the location', default=<class '__main__.Empty'>, annotation='str'),
 ArgProps(arg_name='start_line', required=True, description='The starting line number', default=<class '__main__.Empty'>, annotation='int'),
 ArgProps(arg_name='end_line', required=True, description='The ending line number', default=<class '__main__.Empty'>, annotation='int'),
 ArgProps(arg_name='optional', required=False, description=None, default=None, annotation='Optional')]

In [211]:
def render_arg_props(arg_props: ArgProps) -> Dict[str, str]:
    arg_props_templates = {
        'annotate': '{arg_name}: {annotation}',
        'self_assign': 'self._{arg_name} = {arg_name}',
        'dict_assign': '"{arg_name}": {arg_name}',
        'generate_property': '''\
@property
def {arg_name}(self) -> {annotation}:
    """
    {description}
    """
    return self._{arg_name}
''',
        'generate_setter': '''\
@file_path.setter
def {arg_name}(self, value: {annotation}) -> None:
    self._{arg_name} = value
    self._data["{arg_name}"] = value
''',
        'generate_overload': '''\
@overload
def __getitem__(self, key: Literal["{arg_name}"]) -> {annotation}: ...
'''
    }

    rendered = {}

    for key, value in arg_props_templates.items():
        rendered[key] = value.format(**arg_props.model_dump())
    
    return rendered

In [212]:
rendered = render_arg_props(arg_props_list[0])
rendered

{'annotate': 'file_path: str',
 'self_assign': 'self._file_path = file_path',
 'dict_assign': '"file_path": file_path',
 'generate_property': '@property\ndef file_path(self) -> str:\n    """\n    The file path of the location\n    """\n    return self._file_path\n',
 'generate_setter': '@file_path.setter\ndef file_path(self, value: str) -> None:\n    self._file_path = value\n    self._data["file_path"] = value\n',
 'generate_overload': '@overload\ndef __getitem__(self, key: Literal["file_path"]) -> str: ...\n'}

``` python


file_path: str

self._file_path = file_path

"file_path": file_path,


@property
def file_path(self) -> str:
    """
    Get the file path.
    """
    return self._file_path


@file_path.setter
def file_path(self, value: str) -> None:
    """
    Set the file path.
    """
    self._file_path = value
    self._data["file_path"] = value

@overload
def __getitem__(self, key: Literal["file_path"]) -> str: ...

Literal["file_path", "start_line", "end_line"]

```

In [187]:
from typing import TypedDict, Any

class LocationData(TypedDict):
    file_path: str
    start_line: int
    end_line: int

class Location:
    def __init__(self, *, file_path: str, start_line: int, end_line: int):
        self._file_path = file_path
        self._start_line = start_line
        self._end_line = end_line
        self._data: LocationData = {
            "file_path": file_path,
            "start_line": start_line,
            "end_line": end_line
        }


    @property
    def file_path(self) -> str:
        """
        Get the file path.
        """
        return self._file_path

    @file_path.setter
    def file_path(self, value: str) -> None:
        """
        Set the file path.
        """
        self._file_path = value
        self._data["file_path"] = value

    @property
    def start_line(self) -> int:
        """
        Get the start line.
        """
        return self._start_line

    @start_line.setter
    def start_line(self, value: int) -> None:
        """
        Set the start line.
        """
        self._start_line = value
        self._data["start_line"] = value

    @property
    def end_line(self) -> int:
        """
        Get the end line.
        """
        return self._end_line

    @end_line.setter
    def end_line(self, value: int) -> None:
        """
        Set the end line.
        """
        self._end_line = value
        self._data["end_line"] = value

    @overload
    def __getitem__(self, key: Literal["file_path"]) -> str: ...

    @overload
    def __getitem__(self, key: Literal["start_line"]) -> int: ...

    @overload
    def __getitem__(self, key: Literal["end_line"]) -> int: ...
    

    def __getitem__(self, key: Literal["file_path", "start_line", "end_line"]): # -> Union[str, int]:
        return self._data[key]

    def __setitem__(self, key: Literal["file_path", "start_line", "end_line"], value) -> None:
        if key in self._data:
            setattr(self, key, value)
        else:
            raise KeyError(f"'{key}' is not a valid attribute for Location")

    def __setitem__(self, key: str, value: Any) -> None:
        if key in self._data:
            setattr(self, key, value)
        else:
            raise KeyError(f"'{key}' is not a valid attribute for Location")

    def __repr__(self) -> str:
        return f"Location({self._data})"

    def dict(self) -> LocationData:
        return self._data.copy()

# 사용 예시
location = Location(file_path="/path/to/file.py", start_line=10, end_line=20)


location.end_line = 5

location["end_line"]

5

In [91]:
class LocationData(TypedDict):
    file_path: str = "hi"
    start_line: int
    
location = LocationData()
location

{}

In [89]:
## Shorter

from typing import TypedDict, Any

class RequiredHolder:
    pass

class LocationData(TypedDict):
    file_path: str = "hi"
    start_line: int

class Location:
    def __init__(self, file_path: str = "some_path", start_line: int = RequiredHolder):
        self._file_path: str = file_path
        self._start_line: int = start_line
        self._data: LocationData = {
            "file_path": file_path,
            "start_line": start_line,
        }

    @property
    def file_path(self) -> str:
        """
        Get the file path.
        """
        return self._file_path

    @property
    def start_line(self) -> int:
        """
        Get the start line.
        """
        return self._start_line

    @file_path.setter
    def file_path(self, value: str) -> None:
        self._file_path = value
        self._data["file_path"] = value

    @start_line.setter
    def start_line(self, value: int) -> None:
        self._start_line = value
        self._data["start_line"] = value

    @overload
    def __getitem__(self, key: Literal["file_path"]) -> str: ...

    @overload
    def __getitem__(self, key: Literal["start_line"]) -> int: ...

    def __getitem__(self, key: Literal["file_path", "start_line"]) -> Union[str, int]:
        return self._data[key]

    def __setitem__(self, key: Literal["file_path", "start_line"], value: Union[str, int]) -> None:
        if key in self._data:
            setattr(self, key, value)
        else:
            raise KeyError(f"'{key}' is not a valid attribute for Location")

    def __repr__(self) -> str:
        return f"Location({self._data})"

    def dict(self) -> LocationData:
        return self._data.copy()


In [90]:
location = Location(file_path="/path/to/file.py")

location["start_line"]

__main__.RequiredHolder

In [76]:
location['file_path']

'/path/to/file.py'

In [77]:
location['file_path'] = "new_path"

In [79]:
location['file_path'], location.file_path

('new_path', 'new_path')

In [57]:
class LocationData(TypedDict):
    file_path: str
    start_line: int
    end_line: int


class Location:
    _file_path: str
    start_line: int
    end_line: int

    def __init__(self, file_path: str, start_line: int, end_line: int):
        self.file_path = file_path
        self.start_line = start_line
        self.end_line = end_line
        self._data: LocationData = {
            "file_path": file_path,
            "start_line": start_line,
            "end_line": end_line
        }

    @property
    def file_path(self):
        """
        With Docstring.
        """
        return self._file_path

    def __getitem__(self, key: Literal["file_path", "start_line", "end_line"]) -> Any:
        return self._data[key]

    def __setitem__(self, key: str, value: Any) -> None:
        setattr(self, key, value)

    def __repr__(self) -> str:
        return f"Location({self._data})"

    def dict(self) -> LocationData:
        return self._data


location = Location(file_path="/path/to/file.py", start_line=10, end_line=20)



In [62]:
from pydantic import BaseModel, Field
from typing import Dict, Any

class Location_(BaseModel):
    file_path: str = Field(..., description="The file path of the location")
    start_line: int = Field(..., ge=1, description="The starting line number")
    end_line: int = Field(..., ge=1, description="The ending line number")

In [63]:
dict_assign_template = r"""
"\\[arg_name\\]" = \\[arg_name\\],
"""[2:]

In [ ]:
dict_assign_t = r"""
"\\[arg_name\\]" = \\[arg_name\\],
"""[2:]

arg_and_type_t = r"""
\\[arg_name\\] = \\[type\\],
"""[2:]


template = r"""
class \[data_name\]Data(TypedDict):
    \{arg_and_type_formatted\}


class Location:
    \{arg_and_type_formatted\}

    def __init__(self,
        \{arg_and_type_formatted\}
    ):
        \{arg_assign_formatted\}

        self._data: \[data_name\]Data = {
            \{dict_assign_formatted\}
        }

    def __getitem__(self, key: str) -> Any:
        return self._data[key]

    def __setitem__(self, key: str, value: Any) -> None:
        setattr(self, key, value)

    def __repr__(self) -> str:
        return f"\[data_name\]({self._data})"

    def dict(self) -> \[data_name\]Data:
        return self._data
"""

In [61]:
location = Location(file_path="/path/to/file.py", start_line=10, end_line=20)

location

Location({'file_path': '/path/to/file.py', 'start_line': 10, 'end_line': 20})

In [56]:
from typing import Dict, Any
from typing_extensions import TypedDict





In [22]:
import ast

class CodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.classes = []
        self.functions = []

    def visit_ClassDef(self, node):
        self.classes.append({
            'name': node.name,
            'lineno': node.lineno,
            'end_lineno': node.end_lineno,
            'node': node
        })
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        self.functions.append({
            'name': node.name,
            'lineno': node.lineno,
            'end_lineno': node.end_lineno,
            'node': node
        })
        self.generic_visit(node)

def analyze_code(code):
    tree = ast.parse(code)
    analyzer = CodeAnalyzer()
    analyzer.visit(tree)
    return analyzer.classes, analyzer.functions

# 예제 코드
sample_code = """
class MyClass:
    pass

class MyClass:
    def __init__(self):
        pass

    def my_method(self):
        print("Hello")

def my_function():
    return "World"

another_function = lambda x: x * 2
"""

classes, functions = analyze_code(sample_code)

print("Classes:")
for cls in classes:
    print(f"  {cls['name']} (lines {cls['lineno']}-{cls['end_lineno']})")

print("\nFunctions:")
for func in functions:
    print(f"  {func['name']} (lines {func['lineno']}-{func['end_lineno']})")

# 출력:
# Classes:
#   MyClass (lines 2-7)
#
# Functions:
#   __init__ (lines 3-4)
#   my_method (lines 6-7)
#   my_function (lines 9-10)

Classes:
  MyClass (lines 2-3)
  MyClass (lines 5-10)

Functions:
  __init__ (lines 6-7)
  my_method (lines 9-10)
  my_function (lines 12-13)


In [39]:
import importlib.util
import sys

def import_module_from_path(absolute_path):
    module_name = absolute_path.split("/")[-1].split(".")[0]  # 파일 이름에서 확장자를 제외한 부분을 모듈 이름으로 사용
    spec = importlib.util.spec_from_file_location(module_name, absolute_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

my_module = import_module_from_path(location['file_path'])

In [42]:
my_module.__dict__['Path']

script.Path

In [46]:
location

{'file_path': '/home/crimson/Projects/manager/crimson/folder-syncer/folder-sync-dev/linkable_types/script.py',
 'start_line': 4,
 'end_line': 7}

In [ ]:
def get_annotation(location):
    module = import_module_from_path(location['file_path'])

In [43]:
inspect.getsource(my_module)

'from typing import SupportsIndex\n\n\nclass Path(str):\n    """\n    It is a path.\n    """\n\n\nclass File(str):\n    """\n    This is file.\n    """\n\n\nclass ClassWithLink:\n    """\n    A docs with link.\n\n        [@path]{Path}\n\n    {path Path_}\n    """\n\n    def function(path: Path) -> Path:\n        return path\n'

In [ ]:
def get_location_with_text(file, object_name):
    return object_name

In [17]:
inspect.getfile(Path)

'/home/crimson/Projects/manager/crimson/folder-syncer/folder-sync-dev/linkable_types/script.py'

In [12]:
link_map = extract_yaml_blocks(doc_with_link)[0]

link_map

{'link_map': {'path': 'Path', 'file': 'File'}}

In [7]:
link_map

'class Path(str):\n    """\n    It is a path.\n    """\n'

In [8]:
class IntelliType2:
    pass